In [1]:
!pip install -U vllm==0.10.0
!pip install pylatexenc -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 

# 1. First Step (DeepMath-Zero-Math-7B)

In [2]:
%%writefile first_step.py

def main():
    import os, math, numpy as np
    os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
    SEED = 777
    
    import random
    import numpy as np
    import os
    import torch
    
    def set_seed(seed: int):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        
        os.environ['PYTHONHASHSEED'] = str(seed)
    
    set_seed(SEED)
    
    import vllm
    import torch
    
    llm = vllm.LLM(
        "zwhe99/DeepMath-Zero-Math-7B",
        # "Qwen/Qwen3-8B",
        # "zwhe99/DeepMath-Omn-1.5B",
        # quantization="bitsandbytes",
        tensor_parallel_size=2, 
        gpu_memory_utilization=0.95, 
        trust_remote_code=True,
        dtype="half",
        # dtype = torch.bfloat16,
        enforce_eager=True,
        max_model_len=4096,
        #distributed_executor_backend="ray",
    )
    
    import pandas as pd
    test_df = pd.read_csv('/kaggle/input/aiijc-math/submission (20).csv')
    sub = pd.read_csv('/kaggle/input/aiijc-math/submission (20).csv')
    
    prompt = '''Please reason step by step, and put your final answer within \\boxed{}.'''
    
    def get_info(task):
        if 'парт' in task:
            return ". Размер партии - 100."
        return ''
    
    
    dataset = [
        [{"role": "system", "content": prompt},
        {"role": "user", "content": task + get_info(task)}] for task in test_df['task'].values]
    
    tokenizer = llm.get_tokenizer()
    def formatting(dataset):
        texts = []
        for i in range(len(dataset)):
            texts.append(tokenizer.apply_chat_template(dataset[i], tokenize=False, add_generation_prompt=False))
        return texts
    
    all_prompts = formatting(dataset)
    
    N = 7
    
    responses = llm.generate(
        all_prompts,
        vllm.SamplingParams(
            n=N,  # Number of output sequences to return for each prompt.
            top_p=0.95,  # Float that controls the cumulative probability of the top tokens to consider.
            temperature=0.6,  # randomness of the sampling
            seed=SEED, # Seed for reprodicibility
            skip_special_tokens=True,  # Whether to skip special tokens in the output.
            max_tokens=4096,
        ),
        use_tqdm = True
    )
    
    import regex  # Установите через: pip install regex
    
    def extract_boxed_content(text):
        try:
            pattern = r'\\boxed({(?:[^{}]|(?1))*})'  # Рекурсивный паттерн
            matches = regex.findall(pattern, text)
            return [match[1:-1] for match in matches][-1]  # Удаляем внешние скобки
        except:
            return ''
    
    def get_answer(response):
        answers = []
        for i in range(N):
            ans = extract_boxed_content(response.outputs[i].text)
            if ans != '':
                answers.append(ans)
        if len(answers) == 0:
            answers.append('')
        return pd.Series(answers).mode()[0]
    
    preds = [get_answer(responses[i]) for i in range(len(responses))]
    
    def format_answer(task, ans):
        if 'and' in ans:
            return ans.replace(' \\text{ and } ', '; ')
        if 'pmatrix' in ans:
            return LatexNodes2Text().latex_to_text(ans)[1:-1].strip()
        if ';' in task or 'matr' in task or 'матр' in task:
            return LatexNodes2Text().latex_to_text(ans).replace(',', ';')
        if 'frac' in ans:
            return LatexNodes2Text().latex_to_text(ans).replace(' \\text{ and } ', '; ')
        else:
            return ans.replace(' \\text{ and } ', '; ')
    
    from pylatexenc.latex2text import LatexNodes2Text
    
    sub['answer'] = [f'[{format_answer(task, x)}]' for task, x in zip(sub['task'], preds)]
    
    solutions = []
    for i in range(len(sub)):
        for k in range(N):
            ans = format_answer(sub['task'].iloc[i], extract_boxed_content(responses[i].outputs[k].text))
            if f"[{ans}]" == sub['answer'].iloc[i]:
                solutions.append(responses[i].outputs[k].text)
                break
    
    sub.loc[['век' in x for x in sub['task']], 'answer'] = sub.loc[['век' in x for x in sub['task']], 'answer'].str.replace('(', '').str.replace(')', '').str.replace(',', ';')
    sub.loc[['(' not in x for x in sub['answer']], 'answer'] = sub.loc[['(' not in x for x in sub['answer']], 'answer'].str.replace(',', ';')
    
    sub.to_csv('sub_deepmath-7b_n7.csv', index=False)
    sub['llm_solution'] = solutions
    sub.to_csv('deepmath-7b_solutions.csv', index=False)

if __name__ == '__main__':
    main()

Writing first_step.py


In [3]:
!python first_step.py

2025-09-13 14:59:27.176219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757775567.358226     148 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757775567.410858     148 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO 09-13 14:59:42 [__init__.py:235] Automatically detected platform cuda.
config.json: 100%|█████████████████████████████| 796/796 [00:00<00:00, 4.97MB/s]
WARNING 09-13 14:59:59 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 09-13 14:59:59 [config.py:1604] Using max model len 4096
WARNING 09-13 14:59:59 [arg_utils.py:1690] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 09-13

# 2. Second Step (Qwen3-8B) | LLM-as-judge

In [ ]:
import os, math, numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
SEED = 777

In [ ]:
import random
import numpy as np
import os
import torch

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(SEED)

In [ ]:
import vllm
import torch

llm = vllm.LLM(
    # "zwhe99/DeepMath-Zero-Math-7B",
    "Qwen/Qwen3-8B",
    # "zwhe99/DeepMath-Omn-1.5B",
    # quantization="bitsandbytes",
    tensor_parallel_size=2, 
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half",
    # dtype = torch.bfloat16,
    enforce_eager=True,
    max_model_len=16000,
    #distributed_executor_backend="ray",
)

In [ ]:
import pandas as pd
try:
    test_df = pd.read_csv('deepmath-7b_solutions.csv')
except:
    test_df = pd.read_csv('/kaggle/input/math-aiijc-subs/deepmath-7b_solutions.csv')
sub = pd.read_csv('/kaggle/input/aiijc-math/submission (20).csv')

In [ ]:
user_prompt = "You have been given a student's solution. Check it out step by step. Fix mistakes if there are any. All answers must be given. Put all the answers in one \\boxed{} separating the ';'. Put the final correct answer/answers in \\boxed{}"

In [ ]:
def get_info(task):
    if 'парт' in task:
        return ". Размер партии - 100."
    return ''

In [ ]:
dataset = [
    [{"role": "system", "content": user_prompt},
    {"role": "user", "content": "Task:\n" + task + get_info(task) + "\nStudent Solution:\n" + sol}] for task, sol in test_df[['task', 'llm_solution']].values]

In [ ]:
tokenizer = llm.get_tokenizer()
def formatting(dataset):
    texts = []
    for i in range(len(dataset)):
        texts.append(tokenizer.apply_chat_template(dataset[i], tokenize=False, add_generation_prompt=False))
    return texts

In [ ]:
all_prompts = formatting(dataset)

In [ ]:
N = 7

responses = llm.generate(
    all_prompts,
    vllm.SamplingParams(
        n=N,  # Number of output sequences to return for each prompt.
        top_p=0.95,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0.6,  # randomness of the sampling
        seed=SEED, # Seed for reprodicibility
        skip_special_tokens=True,  # Whether to skip special tokens in the output.
        max_tokens=16000,
    ),
    use_tqdm = True
)

In [ ]:
import regex  # Установите через: pip install regex

def extract_boxed_content(text):
    try:
        pattern = r'\\boxed({(?:[^{}]|(?1))*})'  # Рекурсивный паттерн
        matches = regex.findall(pattern, text)
        return [match[1:-1] for match in matches][-1]  # Удаляем внешние скобки
    except:
        return ''

def get_answer(response):
    answers = []
    for i in range(N):
        ans = extract_boxed_content(response.outputs[i].text)
        if ans != '':
            answers.append(ans)
    if len(answers) == 0:
        answers.append('')
    return pd.Series(answers).mode()[0]


In [ ]:
preds = [get_answer(responses[i]) for i in range(len(responses))]

In [ ]:
def format_answer(task, ans):
    if 'and' in ans:
        return ans.replace(' \\text{ and } ', '; ')
    if 'pmatrix' in ans:
        return LatexNodes2Text().latex_to_text(ans)[1:-1].strip()
    if ';' in task or 'matr' in task or 'матр' in task:
        return LatexNodes2Text().latex_to_text(ans).replace(',', ';')
    if 'frac' in ans:
        return LatexNodes2Text().latex_to_text(ans).replace(' \\text{ and } ', '; ')
    else:
        return ans.replace(' \\text{ and } ', '; ')

In [ ]:
!pip install pylatexenc -qq
from pylatexenc.latex2text import LatexNodes2Text

In [ ]:
sub['answer'] = [f'[{format_answer(task, x)}]' for task, x in zip(sub['task'], preds)]

In [ ]:
sub.loc[['век' in x for x in sub['task']], 'answer'] = sub.loc[['век' in x for x in sub['task']], 'answer'].str.replace('(', '').str.replace(')', '').str.replace(',', ';')
sub.loc[['(' not in x for x in sub['answer']], 'answer'] = sub.loc[['(' not in x for x in sub['answer']], 'answer'].str.replace(',', ';')

In [ ]:
sub.to_csv('sub_deepmath-7b_n7__qwen3-8b-as-judge-n7.csv', index=False)

In [ ]:
i = 0
k = 0
print("Task:\n" + test_df['task'].iloc[i])
print("Solution:\n" + responses[i].outputs[k].text)